In [209]:
import sys
from pathlib import Path

repo_dir = Path('..').resolve()
assert repo_dir.name == 'hicetnunc-dataset', repo_dir
if str(repo_dir) not in sys.path:
    sys.path.append(str(repo_dir))

import src.reload; src.reload.reload()

from tqdm import tqdm
from collections import Counter


nft_state_log = src.utils.read_json(src.config.nft_state_log_file)
nft_state = src.contracts.nft_state.NFTState()
nft_state_replayer = src.contracts.state_utils.StateReplayer(nft_state_log, nft_state)

ah_state_log = src.utils.read_json(src.config.ah_state_log_file)
ah_state = src.contracts.art_house_state.ArtHouseState()
ah_state_replayer = src.contracts.state_utils.StateReplayer(ah_state_log, ah_state)

money_state_log = src.utils.read_json(src.config.money_state_log_file)
money_state = src.contracts.money_state.MoneyState()
money_state_replayer = src.contracts.state_utils.StateReplayer(money_state_log, money_state)

tr_info_db = src.tr.info_db.TrInfoDB()

nft_state_replayer.replay_to_end()
ah_state_replayer.replay_to_end()
money_state_replayer.replay_to_end()

In [210]:
tokens_db = {}
swaps_db = {}

assert set(nft_state.tokens.keys()) == set(ah_state.tokens.keys())

for token_id, nft_token_info in nft_state.tokens.items():
    tokens_db[str(token_id)] = {
        'token_id': token_id,
        'creator': nft_token_info['creator'],
        'info_ipfs': nft_token_info['info_ipfs'],
        'mint_count': nft_token_info['mint_count'],
        'mint_row_id': nft_token_info['mint_row_id'],
    }

for token_id, ah_token_info in tqdm(ah_state.tokens.items()):
    db_entry = tokens_db[str(token_id)]
    db_entry['royalties'] = ah_token_info['royalties']
    db_entry['mint_ah_row_id'] = ah_token_info['mint_ah_row_id']

    assert db_entry['creator'] == ah_token_info['creator']
    assert db_entry['mint_count'] == ah_token_info['mint_count']
    assert db_entry['info_ipfs'] == ah_token_info['info_ipfs']

    nft_mint_tr = tr_info_db.get_full_transaction_by_row_id(db_entry['mint_row_id'])
    ah_mint_tr = tr_info_db.get_full_transaction_by_row_id(ah_token_info['mint_ah_row_id'])
    assert nft_mint_tr == ah_mint_tr

for swap_id, ah_swap_info in ah_state.swaps.items():
    assert str(ah_swap_info['token_id']) in tokens_db
    swaps_db[str(swap_id)] = {
        'swap_id': swap_id,
        'created_row_id': ah_swap_info['created_row_id'],
        'token_id': ah_swap_info['token_id'],
        'price': ah_swap_info['price'],
        'initial_count': ah_swap_info['initial_count'],
    }

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12942/12942 [00:06<00:00, 1891.89it/s]


In [211]:
token_info_fields_cnt = Counter()

for token_id, token_db_entry in tqdm(tokens_db.items()):
    info_ipfs_fpath = get_ipfs_fpath(token_db_entry['info_ipfs'], 'ipfs0')
    token_info = src.utils.read_json(info_ipfs_fpath)

    for field in token_info:
        token_info_fields_cnt[field] += 1
    token_info_keys = set(token_info.keys())
    assert token_info_keys.issubset({
        'artifactUri', 'creators', 'decimals', 'description', 'displayUri', 'formats',
        'isBooleanAmount', 'name', 'shouldPreferSymbol', 'symbol', 'tags', 'thumbnailUri',
    })

    if int(token_id) == 152:
        assert 'isBooleanAmount' not in token_info
    elif int(token_id) <= 352:
        assert token_info['isBooleanAmount'] == (token_db_entry['mint_count'] == 1)
    else:
        assert token_info['isBooleanAmount'] is False

    if int(token_id) <= 154:
        assert 'shouldPreferSymbol' not in token_info
    else:
        assert token_info['shouldPreferSymbol'] is False

    if 'displayUri' in token_info:
        assert int(token_id) >= 11000
        if token_info['displayUri'] is not None:
            assert validate_ipfs_uri(token_info['displayUri'])

    assert token_info['decimals'] == 0
    assert token_info['thumbnailUri'] == 'ipfs://QmNrhZHUaEqxhyLfqoq1mtHSipkWHeT31LNHb1QEbDHgnc'
    assert token_info['symbol'] == 'OBJKT'

    assert len(token_info['formats']) == 1
    assert token_info['formats'][0]['uri'] == token_info['artifactUri']
    assert validate_ipfs_uri(token_info['formats'][0]['uri'])
    assert len(token_info['creators']) == 1

    for tag in token_info['tags']:
        for ch in [' ', '\n', '\t']:
            assert ch not in tag, (ch, tag)

    if token_info['creators'][0] != token_db_entry['creator']:
        assert token_id == '5571' or token_info['creators'][0] in ['', None]

    token_db_entry['artifact_mime'] = token_info['formats'][0]['mimeType']
    token_db_entry['artifact_ipfs'] = token_info['formats'][0]['uri']
    token_db_entry['meta_creator'] = str(token_info['creators'][0])
    token_db_entry['display_uri_ipfs'] = token_info.get('displayUri', '') or ''
    token_db_entry['tags'] = '\n'.join(token_info['tags'])
    token_db_entry['name'] = token_info['name']
    token_db_entry['description'] = token_info['description']

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12942/12942 [00:04<00:00, 2997.78it/s]


In [212]:
not_in_cache_count = 0
for token_id, token_db_entry in tqdm(tokens_db.items()):
    try:
        artifact_fpath = get_ipfs_fpath(token_db_entry['artifact_ipfs'], 'ipfs1')
        token_db_entry['artifact_file_size'] = artifact_fpath.stat().st_size
    except IpfsNotCachedException:
        not_in_cache_count += 1

    if token_db_entry['display_uri_ipfs'] != '':
        try:
            display_uri_fpath = get_ipfs_fpath(token_db_entry['display_uri_ipfs'], 'ipfs1')
            token_db_entry['display_uri_file_size'] = display_uri_fpath.stat().st_size
        except IpfsNotCachedException:
            not_in_cache_count += 1
    else:
        token_db_entry['display_uri_file_size'] = -1

not_in_cache_count

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12942/12942 [00:01<00:00, 10248.40it/s]


0

In [213]:
for token_id, token_db_entry in tqdm(tokens_db.items()):
    token_db_entry.update(
        tr_info_db.get_tr_info_by_row_id(
            token_db_entry['mint_row_id'],
            prefix='mint_',
        )
    )

for swap_id, swap_db_entry in tqdm(swaps_db.items()):
    swap_db_entry.update(
        tr_info_db.get_tr_info_by_row_id(
            swap_db_entry['created_row_id'],
            prefix='created_',
        )
    )


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13052/13052 [00:07<00:00, 1810.77it/s]


In [214]:
src.utils.write_json(tokens_db, src.config.tokens_db_json_file)
src.utils.write_json(swaps_db, src.config.swaps_db_json_file)

written 9691532 bytes 12942 entries
written 3118873 bytes 13052 entries
